In [1]:
import pandas as pd
import numpy as np
from google.colab import files
#uploaded = files.upload()

In [2]:
Output_1 = pd.read_csv("/content/Output_1.csv", delimiter = ";", decimal= ",")

In [3]:
Output_1

,Image,xMin,yMin,xMax,yMax,Class,Confidence
0,0,407,374,444,407,2,0.690849
1,0,478,347,511,381,2,0.680830
2,0,475,292,500,320,2,0.448830
3,0,313,2,370,47,2,0.401188
4,0,452,414,511,469,2,0.389068
...,...,...,...,...,...,...,...
155295,1718,471,315,507,344,1,0.080164
155296,1718,348,479,385,511,1,0.080058
155297,1718,406,171,436,212,2,0.079998
155298,1718,307,34,358,115,1,0.079976


In [5]:
# We keep the buildings that are predicted with average to high confidence
to_drop = []
images_that_are_dropped = []
for i in range(len(Output_1)):
  if Output_1.Confidence[i] < 0.45:
    to_drop.append(False)
  else:
    to_drop.append(True)
Output_1 = Output_1[to_drop]
Output_1.reset_index(inplace = True)

In [6]:
# Using the metrics to calculate the width, the height and other metrics deriving from them 
xMax = Output_1['xMax']
xMin = Output_1['xMin']
yMax = Output_1['yMax']
yMin = Output_1['yMin']

In [7]:
width, height = xMax - xMin, yMax - yMin

In [8]:
# Converting it from pixels to meters
width = (width*256)/512
height = (height*256)/512

In [9]:
# and then to squared meters
square_meters = width*height

In [10]:
Output_1['square_meters'] = square_meters
Output_1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,Image,xMin,yMin,xMax,yMax,Class,Confidence,square_meters
0,0,0,407,374,444,407,2,0.690849,305.25
1,1,0,478,347,511,381,2,0.680830,280.50
2,300,2,478,91,511,126,2,0.705628,288.75
3,301,2,407,118,444,151,2,0.636602,305.25
4,302,2,478,451,511,494,2,0.591512,354.75


In [11]:
# inhabitants per square meter of footprint
builings_density = 0.065
villas_density = 0.01

In [12]:
# Calculating the population density 
population_density = []
for i in range(len(Output_1)):
    if Output_1['Class'][i] == 1 :
        d = square_meters[i]*builings_density
        population_density.append(d)
    elif Output_1['Class'][i] == 2 :
        d = square_meters[i]*villas_density
        population_density.append(d)

In [13]:
Output_1['population_density'] = population_density
Output_1.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,index,Image,xMin,yMin,xMax,yMax,Class,Confidence,square_meters,population_density
0,0,0,407,374,444,407,2,0.690849,305.25,3.0525
1,1,0,478,347,511,381,2,0.680830,280.50,2.8050
2,300,2,478,91,511,126,2,0.705628,288.75,2.8875
3,301,2,407,118,444,151,2,0.636602,305.25,3.0525
4,302,2,478,451,511,494,2,0.591512,354.75,3.5475


In [14]:
# Figure out how many building of class 1 and class2 are in every image and sum 
# estimated population of the image
from collections import defaultdict
NumObj_Class1 = defaultdict(int)
NumObj_Class2 = defaultdict(int)
Population_d = defaultdict(int)
for i in range(len(Output_1)):
  if Output_1.Class[i] == 1.0 : 
    NumObj_Class1[Output_1.Image[i]] += 1
    NumObj_Class2[Output_1.Image[i]] += 0
  else:
    NumObj_Class2[Output_1.Image[i]] += 1
    NumObj_Class1[Output_1.Image[i]] += 0
  Population_d[Output_1.Image[i]] += Output_1["population_density"][i]

In [15]:
final_output = pd.DataFrame(Output_1["Image"].unique())
final_output["NumObj_Class1"] = list(NumObj_Class1.values())
final_output["NumObj_Class2"] = list(NumObj_Class2.values())
final_output["EstimatedPopulation"] = list(Population_d.values())
final_output.rename(columns = {0: "Image"}, inplace = True)

In [16]:
final_output

,Image,NumObj_Class1,NumObj_Class2,EstimatedPopulation
0,0,0,2,5.85750
1,2,0,8,29.12500
2,8,0,3,9.26500
3,9,0,9,36.49750
4,33,0,33,86.92500
...,...,...,...,...
500,1701,0,8,26.35500
501,1705,0,9,37.39750
502,1706,2,3,315.00250
503,1716,2,2,331.76500
